# Import

In [1]:
from anytree import Node

In [2]:
with open('1.txt') as f:
    lines = f.readlines()

# Description of the data

In [3]:
len(lines)

10005

In [4]:
lines[0] #the number of vertices in the phenotype tree

'19830\n'

In [5]:
n = int(lines[0].rstrip())
n

19830

In [6]:
len(lines[1]) #the parent identifiers for the vertices

84052

In [7]:
parents = [int(i) for i in lines[1].rstrip().split(' ')]

In [8]:
parents[:5]

[1, 1, 1, 1, 1]

In [9]:
len(lines[2]) #information content values of the corresponding vertices

78220

In [10]:
info = [int(i) for i in lines[2].rstrip().split(' ')]

In [11]:
info[:5]

[80, 119, 98, 108, 131]

In [12]:
lines[3] #the number of diseases

'5000\n'

In [13]:
m = int(lines[3].rstrip())
m

5000

In [14]:
len(lines[4:5004]) #contain descriptions of diseases

5000

In [15]:
desc_d = []
for line in lines[4:5004]:
    desc_d.append([int(i) for i in line.rstrip().split(' ')])

In [16]:
desc_d[:2]

[[3, 2915, 5421, 7077], [2, 2067, 2500]]

In [17]:
lines[5004] #the number of patients

'5000\n'

In [18]:
l = int(lines[5004].rstrip())
l

5000

In [19]:
len(lines[5005:10005]) #descriptions of patients

5000

In [20]:
desc_p = []
for line in lines[5005:10005]:
    desc_p.append([int(i) for i in line.rstrip().split(' ')])

In [21]:
desc_p[:3]

[[2, 17474, 18858], [1, 18889], [3, 12673, 15214, 18886]]

In [22]:
lines[10004]

'2 12093 16024\n'

# Create Tree Structure

## Initiate root

In [23]:
root = Node(1)

In [24]:
nodes = {}
nodes[root.name] = root

## Create Tree

In [25]:
%%time

for i, j in zip(range(2,n+1), parents):
    name = i
    nodes[name] = Node(name, parent=nodes[j])

CPU times: user 2.07 s, sys: 8.56 ms, total: 2.08 s
Wall time: 2.08 s


In [26]:
import gc
gc.collect()

59

# Task

## Function to find LCA

In [27]:
def commonancestors(a, b):
    if a.parent == b:
        return b
    elif b.parent == a:
        return a
    ancestors = [node.ancestors for node in [a,b]]
    common = []
    for parentnodes in zip(*ancestors): 
        parentnode = parentnodes[0]
        if all([parentnode is p for p in parentnodes[1:]]):
            common.append(parentnode)
        else:
            break
    return tuple(common)

## IC dictionary

In [28]:
# create info dictionary
infod = {}

In [29]:
for i , j in enumerate(info):
    infod[i+1] = j

# Other variables

In [38]:
# disease score
d_scores = []

In [39]:
# result output
result = []

In [40]:
# create score
score = 0

In [41]:
# create minimum
maxim = 0

# Main code block

In [42]:
from tqdm import tqdm

In [43]:
%%time 

# loop through patients
for i in tqdm(range(1,l+1)):
    d_scores = []
    # loop through diseases
    for j in range(1, m+1):
        score = 0
        # loop through patient descriptions
        # skip first item, which is no. of desc
        for d in desc_p[i-1][1:]:
            maxim = 0
            # loop through disease descriptions:
            # skip first item, same reason as above
            for q in desc_d[j-1][1:]:
                # find LCA(q,d)
                anc = commonancestors(nodes[q],nodes[d])
                # anytree LCA function tweak
                try:
                    if len(anc) > 1:
                        key = anc[-1].name
                    elif len(anc) == 1:
                        key = anc[0].name
                except TypeError as e:
                    key = anc.name                        
                info_ = infod[key]
                # find max(IC(LCA(q,d)))
                if info_ >= maxim:
                    maxim = info_
            score += maxim
        d_scores.append(score)
    disease = d_scores.index(max(d_scores)) +1
    result.append(str(disease) + '\n')

100%|██████████| 5000/5000 [42:40<00:00,  1.95it/s]

CPU times: user 42min 39s, sys: 3.88 s, total: 42min 43s
Wall time: 42min 40s


In [44]:
with open('result1.txt', 'w') as f:
    f.writelines(result)

In [45]:
len(result)

5000